In [169]:
from collections import namedtuple

In [170]:
with open("input.txt", "rt") as f:
    plots = f.read().strip().split("\n")

n_rows = len(plots)
n_cols = len(plots[0])

print(f"{n_rows = }")
print(f"{n_cols = }")

n_rows = 131
n_cols = 131


In [171]:
Point = namedtuple("Point", ["x", "y"])


def up(p: Point, n: int = 1) -> Point:
    return Point(p.x, p.y - n)


def left(p: Point, n: int = 1) -> Point:
    return Point(p.x - n, p.y)


def right(p: Point, n: int = 1) -> Point:
    return Point(p.x + n, p.y)


def down(p: Point, n: int = 1) -> Point:
    return Point(p.x, p.y + n)

In [172]:
start = None
for y, row in enumerate(plots):
    if "S" in row:
        start = Point(row.index("S"), y)
start

Point(x=65, y=65)

# Part 1

In [173]:
def lookup_plot(p: Point) -> str:
    return plots[p.y][p.x]

positions = {start}
for step in range(64):
    next_positions = set()
    while positions:
        position = positions.pop()

        for direction in (up, left, right, down):
            next_position = direction(position)
            if lookup_plot(next_position) in ".S":
                next_positions.add(next_position)

    positions = next_positions

len(positions)

3762

# Part 2

I stole the idea from reddit. Because of empty rows in input data, the amount of final positions can be estimated for any amount of steps by interpolating quadratic equasion. There is a bunch of assmptions here, like the fact that input map has to be a square of size 131 and start is at the exact center.


In [174]:
def lookup_infinite_plot(p: Point) -> str:
    x = p.x % n_cols
    y = p.y % n_rows
    return plots[y][x]

def get_n_of_final_positions(steps):
    positions = {start}
    visited = set()
    final_positions = 0

    for step in range(steps):
        next_positions = set()
        while positions:
            position = positions.pop()

            for direction in (up, left, right, down):
                next_position = direction(position)
                if lookup_infinite_plot(next_position) in ".S" and next_position not in visited:
                    next_positions.add(next_position)
                    visited.add(next_position)
                    if (steps - step - 1) % 2 == 0:
                        final_positions += 1

        positions = next_positions

    return final_positions 

print(f"{get_n_of_final_positions(65) = }")
print(f"{get_n_of_final_positions(65 + 131) = }")
print(f"{get_n_of_final_positions(65 + 131 + 131) = }")

get_n_of_final_positions(65) = 3868
get_n_of_final_positions(65 + 131) = 34368
get_n_of_final_positions(65 + 131 + 131) = 95262


In [175]:
def interpolate(p0: Point, p1: Point, p2: Point, x: int) -> int:
    l0 = (x - p1.x) * (x - p2.x) / ((p0.x - p1.x) * (p0.x - p2.x))
    l1 = (x - p0.x) * (x - p2.x) / ((p1.x - p0.x) * (p1.x - p2.x))
    l2 = (x - p0.x) * (x - p1.x) / ((p2.x - p0.x) * (p2.x - p1.x))

    y = p0.y * l0 + p1.y * l1 + p2.y * l2

    assert y == int(y)

    return int(y)


x0 = 65
x1 = 65 + 131
x2 = 65 + 131 + 131

p0 = Point(x0, get_n_of_final_positions(x0))
p1 = Point(x1, get_n_of_final_positions(x1))
p2 = Point(x2, get_n_of_final_positions(x2))

interpolate(p0, p1, p2, 26501365)

621944727930768